In [ ]:
from pybalmorel import Balmorel
from pybalmorel.utils import symbol_to_df
import matplotlib.pyplot as plt

# Collect results
m = Balmorel('../')
m.collect_results()

In [ ]:
# Interactive Bar Chart Plotting
m.results.interactive_bar_chart()

In [ ]:
import matplotlib.pyplot as plt

aggregation = ['"true" model', '70 clusters', '50 clusters', '30 clusters', '10 clusters']
CPU_time = [91 + 33/60, 13, 5 + 55/60, 1 + 24/60, 40/60]

fig, ax = plt.subplots(figsize=(5, 3))
ax.bar(aggregation, CPU_time, color=[0.8, .2, .2])
ax.set_ylabel('CPU Time [h]')
labels = ax.get_xticklabels()
ax.set_xticklabels(labels, rotation=90)
fig.savefig('cpu_time.png', transparent=True, bbox_inches='tight')

# Investigate Overlap Effect

In [ ]:
# Load Results at High and Low Resolution
dfh = symbol_to_df(m.results.db['N70'], 'PRO_YCRAGFST')
dfm = symbol_to_df(m.results.db['N10'], 'PRO_YCRAGFST')
dfl = symbol_to_df(m.results.db['N2'], 'PRO_YCRAGFST')
# dfh_curt = symbol_to_df(m.results.db['N70'], 'CURT_YCRAFST')
# dfm_curt = symbol_to_df(m.results.db['N10'], 'CURT_YCRAFST')
# dfl_curt = symbol_to_df(m.results.db['N2'], 'CURT_YCRAFST')
dfh_el = symbol_to_df(m.results.db['N70'], 'EL_DEMAND_YCRST')
dfm_el = symbol_to_df(m.results.db['N10'], 'EL_DEMAND_YCRST')
dfl_el = symbol_to_df(m.results.db['N2'], 'EL_DEMAND_YCRST')
dfh_cap = symbol_to_df(m.results.db['N70'], 'G_CAP_YCRAF')
dfm_cap = symbol_to_df(m.results.db['N10'], 'G_CAP_YCRAF')
dfl_cap = symbol_to_df(m.results.db['N2'], 'G_CAP_YCRAF')

In [ ]:
# Load inputs
# scenarios = ['N70', 'N50', 'N30', 'N10', 'N2']
scenarios = ['N70', 'N50']
for SC in scenarios:
    if SC not in m.input_data:
        m.load_incfiles(SC)

fig, axes = plt.subplots(nrows=len(scenarios), figsize=(15, 15))
fig.subplots_adjust(hspace=0.5)
n = 0
for SC in scenarios:
    # Get potential
    df_pot = symbol_to_df(m.input_data[SC], 'SUBTECHGROUPKPOT').query('TECH_GROUP == "WINDTURBINE_OFFSHORE" or TECH_GROUP == "WINDTURBINE_ONSHORE"').pivot_table(columns='CCCRRRAAA', values='Value', aggfunc='sum')    
    # print('Total potential: %0.2f'%df_pot.sum().sum())
    # Get profile
    df_pro = symbol_to_df(m.input_data[SC], 'WND_VAR_T').pivot_table(index=['SSS', 'TTT'], columns='AAA', values='Value')
    df_pro_normed = df_pro / df_pro.max() # Normalise
    
    # Calculate potential production
    prod_pot = df_pro_normed.mul(df_pot.sum())
    
    # Figure out when production is above X % of peak
    fraction_of_peak = 0.5
    idx = prod_pot >= prod_pot.max() * fraction_of_peak
    print('\nFor scenario %s..'%SC)
    # print('Peak production: ', prod_pot[idx][prod_pot.columns[0]].dropna())
    # print('Simultaneous peak production at %0.0f pct: %d h'%(fraction_of_peak*100, len(prod_pot[idx].dropna())))
    # print('Total overlaps at  %0.0f pct: %0.0f hours/region'%(fraction_of_peak*100, prod_pot[idx].count(axis=1).sum()/len(prod_pot.columns)))
    # print('Total non-overlaps at %0.0f pct: %0.0f hours'%(fraction_of_peak*100, len(prod_pot.index)*len(prod_pot.columns) - prod_pot[idx].count(axis=1).sum()))
    # print('Total non-overlaps at %0.0f pct: %0.0f hours/region'%(fraction_of_peak*100, len(prod_pot.index) - prod_pot[idx].count(axis=1).sum()/len(prod_pot.columns)))
    print('Total non-overlapping production at %0.0f pct: %0.0f TWh'%(fraction_of_peak*100, prod_pot[idx].sum().sum()/1e6))
    print('Total non-overlapping production at %0.0f pct: %0.0f TWh/region'%(fraction_of_peak*100, prod_pot[idx].sum().sum()/len(prod_pot.columns)/1e6))
    # print('Total non-overlapping production at %0.0f pct: %0.0f hours/region'%(fraction_of_peak*100, prod_pot[idx].isna().sum().sum()/len(prod_pot.columns)))
    # print('Sum of peak production: ', prod_pot.max().sum(), '\n')
    # print(df_pot.sum())
    
    # The average of maximum production through all geographies, at any point in time
    # (i.e.: What is the maximum possible production of wind that can be transmitted to the rest of the country on average, in any point in time?)
    print('Average yearly power %0.2f MW'%prod_pot.sum(axis=1).mean())
    print('Deviation in yearly power %0.2f MW'%prod_pot.sum(axis=1).std())

    prod_pot.plot.area(ax=axes[n], legend=False)
    axes[n].set_ylim([0, 100000])
    axes[n].set_xlim([0, 8736])
    axes[n].set_title(SC)
    n += 1
    
    
    # Calculate curtailment
    
    ## Production
    pro_actual = symbol_to_df(m.results.db[SC], 'PRO_YCRAGFST').pivot_table(index=['Season', 'Time'], columns='Area', values='Value')
    pot_actual = symbol_to_df(m.results.db[SC], 'G_CAP_YCRAF').pivot_table(index=['Technology'], columns='Area', values='Value').loc[['WIND-ON', 'WIND-OFF']]
    # df_pro_normed * G_CAP - PRO_YCRAGFST
    print(pot_actual)
    print(df_pro_normed)

In [ ]:
fig, axes = plt.subplots(nrows=3, figsize=(10, 7))
n = 0
# ylims = [0, 50e3]
xlims = [0, 1092]
for res in ['h', 'm', 'l']:
    df = locals()['df%s'%res].query('Fuel == "WIND"').pivot_table(index=['Season', 'Time'], values='Value', columns='Region', aggfunc='sum') 
    # curt = locals()['df%s_curt'%res].query('Fuel == "WIND"').pivot_table(index=['Season', 'Time'], values='Value', columns='Region', aggfunc='sum') 
    el = locals()['df%s_el'%res].query('Category == "EXOGENOUS"').pivot_table(index=['Season', 'Time'], values='Value', aggfunc='sum') 
    cap = locals()['df%s_cap'%res].query('Fuel == "WIND"').pivot_table(columns='Region', values='Value', aggfunc='sum') 
    
    # print(df / cap.sum())
    df.plot.area(stacked=True, ax=axes[n], legend=False)
    el.plot(ax=axes[n], color='k', linewidth=1)
    
    print('Residual load for %s resolution'%res)
    print((df.sum(axis=1)-el.Value).sum())
    # print(el)
    
    # axes[n].set_ylim(ylims)
    axes[n].set_xlim(xlims)
    
    n += 1

# Other Transport

## Biomass

In [ ]:
from pybalmorel.utils import symbol_to_df
import gams
import os
import numpy as np 

SC = 'N2'
ws = gams.GamsWorkspace(system_directory='/appl/gams/47.6.0')
db = ws.add_database_from_gdx(os.path.abspath('../%s/model/all_endofmodel.gdx'%SC))

df = symbol_to_df(db, 'VFUELTRANSPORT')
df


# Capacities on Map

In [ ]:
import geopandas as gpd

scenarios = ['N2', 'N10', 'N30', 'N50', 'N70', 'base']

df = m.results.get_result('G_CAP_YCRAF')
for SC in scenarios:
    fig, ax = plt.subplots()
    if SC != 'base':
        gf = gpd.read_file('geofiles/DE-DH-WNDFLH-SOLEFLH_%dcluster_geofile.gpkg'%(int(SC.lstrip('N'))))
    else:
        gf = gpd.read_file('geofiles/municipalities.gpkg')[['Name', 'geometry']]
        gf.columns = ['cluster_name', 'geometry']
    temp = df.query('Scenario == @SC and Technology == "WIND-OFF"')[['Region', 'Value']].pivot_table(index='Region', values='Value', aggfunc='sum').reset_index()
    temp.columns = ['cluster_name', 'Value']
    gf.merge(temp, on='cluster_name').plot(ax=ax, column='Value', legend=True)
    print(gf.merge(temp, on='cluster_name'))
    ax.set_title(SC)

In [ ]:
from pybalmorel import Balmorel

m.results.plot_profile('Electricity', 2050, 'europe')